In [1]:
#!pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import random as rd
import seaborn as sbs
import csv
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from matplotlib import pyplot

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow
from tensorflow import data
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error

In [4]:
file_data = 'DS_Agua_2017_2022.csv'
path_name='data/'
arquivo = f'{path_name}{file_data}'
caminho = 'results/'


In [5]:
dataset = pd.read_csv(arquivo, sep =';', encoding = 'latin1', decimal='.')

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45240 entries, 1 to 45240
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DT_MEDICAO_HORA         45240 non-null  object 
 1   PRECIPITACAO            45240 non-null  float64
 2   PRESSAO_ATMOSFERICA     45240 non-null  float64
 3   TEMPERATURA_DO_AR_C     45240 non-null  float64
 4   UMIDADE_RELATIVA_DO_AR  45240 non-null  float64
 5   VELOCIDADE_VENTO        45240 non-null  float64
 6   VL_MEDICAO              45240 non-null  float64
dtypes: float64(6), object(1)
memory usage: 2.8+ MB


In [7]:
#verifica se existe variáveis nulas 
dataset.isna().sum()

DT_MEDICAO_HORA           0
PRECIPITACAO              0
PRESSAO_ATMOSFERICA       0
TEMPERATURA_DO_AR_C       0
UMIDADE_RELATIVA_DO_AR    0
VELOCIDADE_VENTO          0
VL_MEDICAO                0
dtype: int64

In [8]:
dataset.head(10)

,DT_MEDICAO_HORA,PRECIPITACAO,PRESSAO_ATMOSFERICA,TEMPERATURA_DO_AR_C,UMIDADE_RELATIVA_DO_AR,VELOCIDADE_VENTO,VL_MEDICAO
1,2017-01-01 00:00:00,0.0,1009.5,25.9,75.0,1.3,5016.845991
2,2017-01-01 01:00:00,0.0,1009.6,25.7,76.0,0.9,4998.296902
3,2017-01-01 02:00:00,0.0,1009.2,25.6,76.0,1.1,5013.330096
4,2017-01-01 03:00:00,0.0,1008.4,25.9,77.0,1.7,4972.739746
5,2017-01-01 04:00:00,0.0,1008.1,23.7,85.0,1.7,4921.942265
6,2017-01-01 05:00:00,0.2,1007.5,23.7,87.0,0.4,4838.136258
7,2017-01-01 06:00:00,0.0,1007.3,24.2,88.0,0.5,4833.215005
8,2017-01-01 07:00:00,0.0,1007.3,24.2,87.0,0.7,4795.247653
9,2017-01-01 08:00:00,0.0,1007.7,24.7,85.0,1.0,4763.098501
10,2017-01-01 09:00:00,0.0,1008.5,25.1,83.0,1.2,4715.598748


In [9]:
#tratando o dataframe
df = pd.DataFrame()
df['VL_MEDICAO']=dataset['VL_MEDICAO']
                                
#transformação da série => janela deslizande com 1 passo de tempo
n_time_steps=2
#transformação da série => janela deslizande com 1 passo de tempo
for n_step in range(1,n_time_steps+1,1):
 df['vl-'+str(n_step)]=dataset['VL_MEDICAO'].shift(n_step)  

#transformação da série => janela deslizande com 1 passo de tempo
#n=1
#df['vl-n']=dataset['VL_MEDICAO'].shift(n)
df['tp-n']=dataset['TEMPERATURA_DO_AR_C']
df['pr-n']=dataset['PRESSAO_ATMOSFERICA']
df['vv-n']=dataset['VELOCIDADE_VENTO']
df['ur-n']=dataset['UMIDADE_RELATIVA_DO_AR']
df['ch-n']=dataset['PRECIPITACAO']

df.dropna(inplace=True)
df.head(15)

,VL_MEDICAO,vl-1,vl-2,tp-n,pr-n,vv-n,ur-n,ch-n
3,5013.330096,4998.296902,5016.845991,25.6,1009.2,1.1,76.0,0.0
4,4972.739746,5013.330096,4998.296902,25.9,1008.4,1.7,77.0,0.0
5,4921.942265,4972.739746,5013.330096,23.7,1008.1,1.7,85.0,0.0
6,4838.136258,4921.942265,4972.739746,23.7,1007.5,0.4,87.0,0.2
7,4833.215005,4838.136258,4921.942265,24.2,1007.3,0.5,88.0,0.0
8,4795.247653,4833.215005,4838.136258,24.2,1007.3,0.7,87.0,0.0
9,4763.098501,4795.247653,4833.215005,24.7,1007.7,1.0,85.0,0.0
10,4715.598748,4763.098501,4795.247653,25.1,1008.5,1.2,83.0,0.0
11,4712.059896,4715.598748,4763.098501,26.2,1009.0,1.5,77.0,0.0
12,4657.193811,4712.059896,4715.598748,28.4,1009.5,1.6,67.0,0.0


In [10]:
#Cria os vetores de X e y para terinamento e teste
vl_medicoes = df.iloc[:,0].values

X = df.iloc[0:dataset.shape[0],1: 6 + n_time_steps]

#Definindo os datasets de treinamento e teste
#X_train, X_test, y_train, y_test = train_test_split(X, vl_medicoes, test_size=0.1, random_state= 1)


In [11]:

#Do total de 45239 linhas, foram separadas 75% para treinamento e 25% para teste
#nlinhas = 33929 # 75% do dataset
nlinhas = int(np.round(df.shape[0] *0.75)) # 
X_train = df.iloc[0:nlinhas,1: 6 + n_time_steps]
X_test = df.iloc[nlinhas:X.shape[0],1: 6 + n_time_steps]

y_train = df.iloc[0:nlinhas,0].values
y_test = df.iloc[nlinhas:X.shape[0],0].values


In [12]:
#Script para criar o arquivo de resultdos
def criar_arquivo_resultado(arquivo):
  fields = ['N_Nos_Layer1','N_Nos_Layer2','N_Nos_Layer3','EPOCAS','MSE','RMSE','MAE','MAPE','Duracao']
  with open(f'{caminho}{arquivo}',"w",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerow(fields)

#Script para escrever os resultados do ciclo de treinamento
def salvar_resultados(N_Nos_Layer1, N_Nos_Layer2, N_Nos_Layer3, EPOCAS, MSE, RMSE, MAE, MAPE, Duracao):
  fields = ['N_Nos_Layer1','N_Nos_Layer2','N_Nos_Layer3','EPOCAS','MSE','RMSE','MAE','MAPE','Duracao']
  data = [N_Nos_Layer1, N_Nos_Layer2, N_Nos_Layer3, EPOCAS, MSE, RMSE, MAE, MAPE, Duracao]
  with open(f'{caminho}{arquivo}', "a",newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    #writer.writerow(fields)
    writer.writerow(data)

In [13]:
def treinar_LSTM(N_Nos_Layer1, N_Nos_Layer2,N_Nos_Layer3, EPOCAS):
  train_feature = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1]))
  train_label = np.array(y_train).reshape((y_train.shape[0], 1, 1))

  n_valores_previsao = 1 # Número de dias que serão armazenados para o ciclo futuro.

  # Armazena a hora de início da execução do Treinamento
  Hora_Inicio = time.time()

  batch_size = 4096 # Tamanho de cada lote de processamento usando para o modelo abstrair os padrões
  # EPOCAS é o número de vezes que o conjunto de lotes irá ser processado pelo modelo na fase de treinamento.

  train_data = data.Dataset.from_tensor_slices((train_feature, train_label))
  train_data = train_data.repeat().batch(batch_size, drop_remainder=True)

  
  model = keras.Sequential()
  model.add(layers.LSTM(N_Nos_Layer1, batch_input_shape=(batch_size, train_feature.shape[1], train_feature.shape[2]),  return_sequences=True, activation='relu'))  
  model.add(layers.LSTM(N_Nos_Layer2, batch_input_shape=(N_Nos_Layer1, train_feature.shape[1], train_feature.shape[2]), return_sequences=True, activation='relu'))  
  model.add(layers.Dense(N_Nos_Layer3, activation="relu"))
#  model.add(layers.Dense(N_Nos_Layer4, activation="relu")) #16
  model.add(layers.Dense(1))


  #model = keras.Sequential()
  #model.add(layers.LSTM(N_Nos_Layer1, batch_input_shape=(batch_size, train_feature.shape[1], train_feature.shape[2]), stateful=True, return_sequences=True))  
  #model.add(layers.LSTM(N_Nos_Layer2, batch_input_shape=(N_Nos_Layer1, train_feature.shape[1], train_feature.shape[2]), stateful=True))  
  #model.add(layers.Dense(n_valores_previsao))

  #mean_squared_error
  model.compile(loss='mean_absolute_percentage_error', optimizer='adam', metrics=['mape'])
  model.summary()
  steps_per_epoch = len(train_feature) // batch_size 
  history = model.fit(train_data, epochs=EPOCAS, steps_per_epoch = steps_per_epoch) 

  val_feature = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))
  val_label = np.array(y_test).reshape((y_test.shape[0], 1, 1))


  test_data = data.Dataset.from_tensor_slices((val_feature, val_label))
  test_data = test_data.repeat().batch(batch_size, drop_remainder=True)

  predict = model.predict(test_data, steps=1)
  #predict = model.predict(train_data, steps=1)

  val_label=val_label[0:len(predict)]

  predict = np.array(predict).reshape(4096,1)
  val_label = np.array(val_label).reshape(4096,1)

  # Armazena a hora fim da execução do Treinamento
  Hora_Fim = time.time()

  #Calcula o tempo de duração da execução do treinamento
  Duracao = Hora_Fim - Hora_Inicio

  val_label=val_label[0:len(predict)]

  #Validação dos resultados

  #Calcular o MAPE (Erro médio percentual absoluto)
  MAPE = ((np.mean(np.abs(val_label -predict) / (val_label)))) * 100 
  print("MAPE=",MAPE)

  #Erro médio quadrático  (diferença quadrática média entre os valores estimados e valores reais) - MSE
  MSE = mean_squared_error(val_label, predict)
  #Raiz quadrada do Erro médio - RMSE
  RMSE = np.sqrt(mean_squared_error(val_label, predict))

  #Distância absoluta média ou Erro médio absoluto - MAE
  MAE= median_absolute_error(y_pred=predict, y_true = val_label)

  salvar_resultados(N_Nos_Layer1, N_Nos_Layer2, N_Nos_Layer3, EPOCAS, MSE, RMSE, MAE, MAPE, Duracao)

In [15]:
# Efetuar o processo de Calibração da LSTM, variando os hiperparâmetros
EPOCAS = 250
arquivo='Result_LSTM_Model.csv'
#n_valores_passado = 10
criar_arquivo_resultado(arquivo)
#Laço para variação da Camada Oculta 1
for N_Nos_Layer1 in range (100,6,-2):
  #Laço para variação da Camada Oculta 2
  for N_Nos_Layer2 in range (100,6,-2):
    for N_Nos_Layer3 in range (100,6,-2):
      #Chama a função de treinamento da LSTM com os hiperparâmetros selecionados para cada iteração
      treinar_LSTM(N_Nos_Layer1, N_Nos_Layer2, N_Nos_Layer3, EPOCAS)
      #Laço para variação do  n_valores_passado
      #for n_valores_passado in range (5,20,5):
    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (4096, 1, 100)            43200     
                                                                 
 lstm_1 (LSTM)               (4096, 1, 100)            80400     
                                                                 
 dense (Dense)               (4096, 1, 100)            10100     
                                                                 
 dense_1 (Dense)             (4096, 1, 1)              101       
                                                                 
Total params: 133,801
Trainable params: 133,801
Non-trainable params: 0
_________________________________________________________________
Epoch 1/250
8/8 [==============================] - 2s 80ms/step - loss: 100.6118 - mape: 100.6118
Epoch 2/250
8/8 [==============================] - 1s 77ms/step - loss: 98.2086 - ma